# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nhl_correlation(): 
    # Function to compute win-loss ratio by area
    def compute_wl_ratio_by_area(teams):
        wins = 0
        losses = 0
        for team in teams:
            wins += nhl_df[nhl_df['team'].str.contains(team)]['W'].sum()
            losses += nhl_df[nhl_df['team'].str.contains(team)]['L'].sum()
        return wins/(wins+losses)
    
    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    # Clean up nhl_df
    nhl_df = nhl_df[nhl_df['year'] == 2018]
    nhl_df = nhl_df[~nhl_df['team'].str.contains('Division')]
    # Almost all of the columns are of type 'str', cast them to numeric
    nhl_numeric_columns = nhl_df.columns[1:-1]
    nhl_df[nhl_numeric_columns] = nhl_df[nhl_numeric_columns].apply(pd.to_numeric)

    # Remove [note **] patterns
    cities['NHL'] = cities['NHL'].str.extract('^(?P<Teams>[\w ]+)(?:\[note [\d]\])?')['Teams']
    # Cast population column to numeric
    cities['Population (2016 est.)[8]'] = pd.to_numeric(cities['Population (2016 est.)[8]'])
    # Drop empty teams
    cities['NHL'] = cities['NHL'].replace('—',np.nan)
    cities = cities[~cities['NHL'].isna()]

    # Extract team names per metropolitan area
    cities['NHL'] = cities['NHL'].str.findall('([A-Z][a-z0-9]+ [A-Z][a-z0-9]+)|([A-Z][a-z0-9 ]+)').apply(lambda x: [cg1 if cg2 == '' else cg2 for cg1,cg2 in x])
    cities['Win Loss Ratio'] = cities['NHL'].apply(compute_wl_ratio_by_area)
    
    population_by_region = cities['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = cities['Win Loss Ratio'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nba_correlation():
    def compute_wl_ratio_by_area(teams):
        wins = 0
        losses = 0
        for team in teams:
            wins += nba_df[nba_df['team'].str.contains(team)]['W'].sum()
            losses += nba_df[nba_df['team'].str.contains(team)]['L'].sum()
        return wins/(wins+losses)
    
    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    # Clean up nba_df
    nba_df = nba_df[nba_df['year'] == 2018]
    nba_df = nba_df[~nba_df['team'].str.contains('Division')]
    nba_df = nba_df.replace('—',np.nan)
    # Almost all of the columns are of type 'str', cast them to numeric
    nba_numeric_columns = nba_df.columns[1:-1]
    nba_df[nba_numeric_columns] = nba_df[nba_numeric_columns].apply(pd.to_numeric)

    # Remove [note **] patterns
    cities['NBA'] = cities['NBA'].str.extract('^(?P<Teams>[\w ]+)(?:\[note [\d]\])?')['Teams']
    # Cast population column to numeric
    cities['Population (2016 est.)[8]'] = pd.to_numeric(cities['Population (2016 est.)[8]'])
    # Drop empty teams
    cities['NBA'] = cities['NBA'].replace('—',np.nan)
    cities = cities[~cities['NBA'].isna()]

    # Extract team names per metropolitan area
    cities['NBA'] = cities['NBA'].str.findall('([A-Z0-9][a-z0-9]+ [A-Z0-9][a-z0-9]+)|([A-Z0-9][a-z0-9 ]+)').apply(lambda x: [cg1 if cg2 == '' else cg2 for cg1,cg2 in x])
    cities['Win Loss Ratio'] = cities['NBA'].apply(compute_wl_ratio_by_area)
    population_by_region = cities['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = cities['Win Loss Ratio'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [3]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def mlb_correlation(): 
    def compute_wl_ratio_by_area(teams):
        wins = 0
        losses = 0
        for team in teams:
            wins += mlb_df[mlb_df['team'].str.contains(team)]['W'].sum()
            losses += mlb_df[mlb_df['team'].str.contains(team)]['L'].sum()
        return wins/(wins+losses)
    
    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    # Clean up mlb df
    mlb_df = mlb_df[mlb_df['year'] == 2018]
    mlb_df = mlb_df[~mlb_df['team'].str.contains('Division')]
    mlb_df = mlb_df.replace('—',np.nan)
    mlb_df = mlb_df.replace('--',np.nan)
    # Almost all of the columns are of type 'str', cast them to numeric
    mlb_numeric_columns = mlb_df.columns[1:-1]
    mlb_df[mlb_numeric_columns] = mlb_df[mlb_numeric_columns].apply(pd.to_numeric)

    # Remove [note **] patterns
    cities['MLB'] = cities['MLB'].str.extract('^(?P<Teams>[\w ]+)(?:\[note [\d]\])?')['Teams']
    # Cast population column to numeric
    cities['Population (2016 est.)[8]'] = pd.to_numeric(cities['Population (2016 est.)[8]'])
    # Drop empty teams
    cities['MLB'] = cities['MLB'].replace('—',np.nan)
    cities = cities[~cities['MLB'].isna()]

    # Extract team names per metropolitan area
    cities['MLB'] = cities['MLB'].str.findall('([A-Z0-9][a-z0-9]+ [A-Z0-9][a-z0-9]+)|([A-Z0-9][a-z0-9 ]+)').apply(lambda x: [cg1 if cg2 == '' else cg2 for cg1,cg2 in x])
    cities['Win Loss Ratio'] = cities['MLB'].apply(compute_wl_ratio_by_area)
    population_by_region = cities['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = cities['Win Loss Ratio'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [4]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nfl_correlation(): 
    def compute_wl_ratio_by_area(teams):
        wins = 0
        losses = 0
        for team in teams:
            wins += nfl_df[nfl_df['team'].str.contains(team)]['W'].sum()
            losses += nfl_df[nfl_df['team'].str.contains(team)]['L'].sum()
        return wins/(wins+losses)
    
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    # Clean up nfl df
    nfl_df = nfl_df[nfl_df['year'] == 2018]
    nfl_df = nfl_df[~((nfl_df['team'].str.contains('AFC'))|(nfl_df['team'].str.contains('NFC')))]
    nfl_df = nfl_df.replace('—',np.nan)
    nfl_df = nfl_df.replace('--',np.nan)
    # Almost all of the columns are of type 'str', cast them to numeric
    nfl_numeric_columns = list(nfl_df.columns[:2]) + list(nfl_df.columns[3:-2])
    nfl_df[nfl_numeric_columns] = nfl_df[nfl_numeric_columns].apply(pd.to_numeric)

    # Remove [note **] patterns
    cities['NFL'] = cities['NFL'].str.extract('^(?P<Teams>[\w ]+)(?:\[note [\d]\])?')['Teams']
    # Cast population column to numeric
    cities['Population (2016 est.)[8]'] = pd.to_numeric(cities['Population (2016 est.)[8]'])
    # Drop empty teams
    cities['NFL'] = cities['NFL'].replace('—',np.nan)
    cities = cities[~cities['NFL'].isna()]

    # Extract team names per metropolitan area

    cities['NFL'] = cities['NFL'].str.findall('([A-Z0-9][a-z0-9]+ [A-Z0-9][a-z0-9]+)|([A-Z0-9][a-z0-9 ]+)').apply(lambda x: [cg1 if cg2 == '' else cg2 for cg1,cg2 in x])
    cities['Win Loss Ratio'] = cities['NFL'].apply(compute_wl_ratio_by_area)
    population_by_region = cities['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = cities['Win Loss Ratio'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]


## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [5]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    mlb_df=pd.read_csv("assets/mlb.csv")
    nhl_df=pd.read_csv("assets/nhl.csv")
    nba_df=pd.read_csv("assets/nba.csv")
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    def nhl_compute_wl_ratio_by_area(teams):
        wins = 0
        losses = 0
        for team in teams:
            wins += nhl_df[nhl_df['team'].str.contains(team)]['W'].sum()
            losses += nhl_df[nhl_df['team'].str.contains(team)]['L'].sum()
        return wins/(wins+losses) if wins + losses != 0 else np.nan
    
    def nba_compute_wl_ratio_by_area(teams):
        wins = 0
        losses = 0
        for team in teams:
            wins += nba_df[nba_df['team'].str.contains(team)]['W'].sum()
            losses += nba_df[nba_df['team'].str.contains(team)]['L'].sum()
        return wins/(wins+losses) if wins + losses != 0 else np.nan
    
    def mlb_compute_wl_ratio_by_area(teams):
        wins = 0
        losses = 0
        for team in teams:
            wins += mlb_df[mlb_df['team'].str.contains(team)]['W'].sum()
            losses += mlb_df[mlb_df['team'].str.contains(team)]['L'].sum()
        return wins/(wins+losses) if wins + losses != 0 else np.nan
    
    def nfl_compute_wl_ratio_by_area(teams):
        wins = 0
        losses = 0
        for team in teams:
            wins += nfl_df[nfl_df['team'].str.contains(team)]['W'].sum()
            losses += nfl_df[nfl_df['team'].str.contains(team)]['L'].sum()
        return wins/(wins+losses) if wins + losses != 0 else np.nan

    # YOUR CODE HERE

    
    # NHL
    nhl_df = nhl_df[nhl_df['year'] == 2018]
    nhl_df = nhl_df[~nhl_df['team'].str.contains('Division')]
    # Almost all of the columns are of type 'str', cast them to numeric
    nhl_numeric_columns = nhl_df.columns[1:-1]
    nhl_df[nhl_numeric_columns] = nhl_df[nhl_numeric_columns].apply(pd.to_numeric)

    # Remove [note **] patterns
    cities['NHL'] = cities['NHL'].str.extract('^(?P<Teams>[\w ]+)(?:\[note [\d]\])?')['Teams']
    
    # Drop empty teams
    cities['NHL'] = cities['NHL'].replace('—',np.nan)

    # Extract team names per metropolitan area
    cities['NHL'] = cities['NHL'].str.findall('([A-Z][a-z0-9]+ [A-Z][a-z0-9]+)|([A-Z][a-z0-9 ]+)').apply(lambda x: [cg1 if cg2 == '' else cg2 for cg1,cg2 in x] if type(x) == type([]) else [])
    cities['NHL Win Loss Ratio'] = cities['NHL'].apply(nhl_compute_wl_ratio_by_area)
    
    
    # NBA
    nba_df = nba_df[nba_df['year'] == 2018]
    nba_df = nba_df[~nba_df['team'].str.contains('Division')]
    nba_df = nba_df.replace('—',np.nan)
    # Almost all of the columns are of type 'str', cast them to numeric
    nba_numeric_columns = nba_df.columns[1:-1]
    nba_df[nba_numeric_columns] = nba_df[nba_numeric_columns].apply(pd.to_numeric)

    # Remove [note **] patterns
    cities['NBA'] = cities['NBA'].str.extract('^(?P<Teams>[\w ]+)(?:\[note [\d]\])?')['Teams']
    # Cast population column to numeric
    cities['Population (2016 est.)[8]'] = pd.to_numeric(cities['Population (2016 est.)[8]'])
    # Drop empty teams
    cities['NBA'] = cities['NBA'].replace('—',np.nan)

    # Extract team names per metropolitan area
    cities['NBA'] = cities['NBA'].str.findall('([A-Z0-9][a-z0-9]+ [A-Z0-9][a-z0-9]+)|([A-Z0-9][a-z0-9 ]+)').apply(lambda x: [cg1 if cg2 == '' else cg2 for cg1,cg2 in x] if type(x) == type([]) else [])
    cities['NBA Win Loss Ratio'] = cities['NBA'].apply(nba_compute_wl_ratio_by_area)
    
    # MLB
    mlb_df = mlb_df[mlb_df['year'] == 2018]
    mlb_df = mlb_df[~mlb_df['team'].str.contains('Division')]
    mlb_df = mlb_df.replace('—',np.nan)
    mlb_df = mlb_df.replace('--',np.nan)
    # Almost all of the columns are of type 'str', cast them to numeric
    mlb_numeric_columns = mlb_df.columns[1:-1]
    mlb_df[mlb_numeric_columns] = mlb_df[mlb_numeric_columns].apply(pd.to_numeric)

    # Remove [note **] patterns
    cities['MLB'] = cities['MLB'].str.extract('^(?P<Teams>[\w ]+)(?:\[note [\d]\])?')['Teams']
    # Drop empty teams
    cities['MLB'] = cities['MLB'].replace('—',np.nan)

    # Extract team names per metropolitan area
    cities['MLB'] = cities['MLB'].str.findall('([A-Z0-9][a-z0-9]+ [A-Z0-9][a-z0-9]+)|([A-Z0-9][a-z0-9 ]+)').apply(lambda x: [cg1 if cg2 == '' else cg2 for cg1,cg2 in x] if type(x) == type([]) else [])
    cities['MLB Win Loss Ratio'] = cities['MLB'].apply(mlb_compute_wl_ratio_by_area)

    # NFL
    nfl_df = nfl_df[nfl_df['year'] == 2018]
    nfl_df = nfl_df[~((nfl_df['team'].str.contains('AFC'))|(nfl_df['team'].str.contains('NFC')))]
    nfl_df = nfl_df.replace('—',np.nan)
    nfl_df = nfl_df.replace('--',np.nan)
    # Almost all of the columns are of type 'str', cast them to numeric
    nfl_numeric_columns = list(nfl_df.columns[:2]) + list(nfl_df.columns[3:-2])
    nfl_df[nfl_numeric_columns] = nfl_df[nfl_numeric_columns].apply(pd.to_numeric)

    # Remove [note **] patterns
    cities['NFL'] = cities['NFL'].str.extract('^(?P<Teams>[\w ]+)(?:\[note [\d]\])?')['Teams']
    # Drop empty teams
    cities['NFL'] = cities['NFL'].replace('—',np.nan)

    # Extract team names per metropolitan area

    cities['NFL'] = cities['NFL'].str.findall('([A-Z0-9][a-z0-9]+ [A-Z0-9][a-z0-9]+)|([A-Z0-9][a-z0-9 ]+)').apply(lambda x: [cg1 if cg2 == '' else cg2 for cg1,cg2 in x] if type(x) == type([]) else [])
    cities['NFL Win Loss Ratio'] = cities['NFL'].apply(nfl_compute_wl_ratio_by_area)

    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    for x in sports:
        for y in sports:
            if x == y: continue
            df1 = cities[[x + ' Win Loss Ratio']].dropna()
            df2 = cities[[y + ' Win Loss Ratio']].dropna()
            df3 = df1.merge(df2,left_index=True,right_index=True)
            p_values.loc[x,y] = stats.ttest_rel(df3[x + ' Win Loss Ratio'],df3[y + ' Win Loss Ratio'])[1]
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values